# MAHTS stats

## Background


## Description



## Getting started


### Load packages

First we import the required Python packages, then we connect to the database, and load the catalog of virtual products.

In [1]:
%matplotlib inline
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

import deacoastlines_statistics as dcl_stats

import os
import sys
import geopandas as gpd
from shapely.geometry import box
from rasterio.transform import array_bounds
import pandas as pd
import shutil

sys.path.append('../Scripts')
from dea_spatialtools import subpixel_contours

## Load in data

In [2]:
# Read in contours
study_area = 7832
output_name = 'multi'
water_index = 'mndwi'
index_threshold = 0.00
baseline_year = '2018'

# Create output folder
output_dir = f'output_data/{output_name}_{study_area}/vectors'
os.makedirs(output_dir, exist_ok=True)
os.makedirs(f'{output_dir}/shapefiles', exist_ok=True)

## Load DEA CoastLines rasters

In [3]:
yearly_ds = dcl_stats.load_rasters(output_name, study_area, water_index)

## Load external data

In [4]:
# Get bounding box to load data for
bbox = gpd.GeoSeries(box(*array_bounds(height=yearly_ds.sizes['y'], 
                                       width=yearly_ds.sizes['x'], 
                                       transform=yearly_ds.transform)), 
                     crs=yearly_ds.crs)

# Estaury mask
estuary_gdf = (gpd.read_file('input_data/estuary_mask.shp', bbox=bbox)
               .to_crs(yearly_ds.crs))

# Rocky shore mask
smartline_gdf = (gpd.read_file('input_data/Smartline.gdb', bbox=bbox)
                 .to_crs(yearly_ds.crs))

# Tide points
points_gdf = (gpd.read_file('input_data/tide_points_coastal.geojson', bbox=bbox)
          .to_crs(yearly_ds.crs))

# Study area polygon
comp_gdf = (gpd.read_file('input_data/50km_albers_grid.shp', bbox=bbox)
            .set_index('id')
            .to_crs(str(yearly_ds.crs)))

# Mask to study area
study_area_poly = comp_gdf.loc[study_area]

# Load climate indices
climate_df = pd.read_csv('input_data/climate_indices.csv', index_col='year')


## Extract shoreline contours

### Extract ocean-masked contours

In [5]:
# Mask dataset to focus on coastal zone only
masked_ds = dcl_stats.contours_preprocess(yearly_ds, 
                                          water_index, 
                                          index_threshold, 
                                          estuary_gdf, 
                                          points_gdf)

# Extract contours
contours_gdf = subpixel_contours(da=masked_ds,
                                 z_values=index_threshold,
                                 min_vertices=10,
                                 dim='year').set_index('year')

Operating in single z-value, multiple arrays mode


## Compute statistics
### Measure distances from baseline

In [6]:
# Extract statistics modelling points along baseline contour
points_gdf = dcl_stats.stats_points(contours_gdf, baseline_year, distance=30)

# Make a copy of the points GeoDataFrame to hold tidal data
tide_points_gdf = points_gdf.copy()

### Compute annual coastline movements

In [7]:
# Calculate annual movements and residual tide heights for every contour
# compared to the baseline year
points_gdf, tide_points_gdf = dcl_stats.annual_movements(yearly_ds, 
                                                         points_gdf, 
                                                         tide_points_gdf, 
                                                         contours_gdf, 
                                                         baseline_year,
                                                         water_index)

### Calculate regressions

In [8]:
points_gdf = dcl_stats.calculate_regressions(yearly_ds, 
                                             points_gdf, 
                                             tide_points_gdf, 
                                             climate_df)

Comparing annual movements with time
Comparing annual movements with tide heights
Comparing annual movements with SOI
Comparing annual movements with IOD
Comparing annual movements with SAM
Comparing annual movements with IPO
Comparing annual movements with PDO


## Export files

In [9]:
# Extract a 50 m buffer around rocky shore features which is used to clip stats
rocky_shore_buffer = dcl_stats.rocky_shores_buffer(smartline_gdf=smartline_gdf, buffer=50)

# Clip stats to study area extent, remove rocky shores and export to GeoJSON
stats_path = f'{output_dir}/stats_{study_area}_{output_name}_{water_index}_{index_threshold:.2f}'
points_gdf = points_gdf[points_gdf.intersects(study_area_poly['geometry'])]
points_gdf = points_gdf[~points_gdf.intersects(rocky_shore_buffer.geometry.unary_union)]
points_gdf.to_file(f'{stats_path}.geojson', driver='GeoJSON')

# Clip annual shoreline contours to study area extent and export to GeoJSON
contour_path = f'{output_dir}/contours_{study_area}_{output_name}_{water_index}_{index_threshold:.2f}'
contours_gdf['geometry'] = contours_gdf.intersection(study_area_poly['geometry'])
contours_gdf.reset_index().to_file(f'{contour_path}.geojson', driver='GeoJSON')

# Export stats and contours as ESRI shapefiles
contour_path = contour_path.replace('vectors', 'vectors/shapefiles')
stats_path = stats_path.replace('vectors', 'vectors/shapefiles')
contours_gdf.to_file(f'{contour_path}.shp')
points_gdf.to_file(f'{stats_path}.shp')

# Create a zip file containing all vector files
shutil.make_archive(base_name=f'output_data/outputs_{study_area}_{output_name}', 
                    format='zip', 
                    root_dir=output_dir)

'/g/data1a/r78/rt1527/dea-notebooks/MAHTS/output_data/outputs_7832_multi.zip'

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** March 2020